HEALTH AND SCORE MECHANISM

In [ ]:
#to add the score and the health and display it

import pygame
import random

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

        self.sprites = []
        self.current_sprite = 0
        for i in range(1, 20):
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Character/Frame {i}.png').convert_alpha(), 
                (100, 150)
            )
            self.sprites.append(sprite)

        self.jump_sprites = []
        for i in range(1, 13):  
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Jump/Jump_frame {i}.png').convert_alpha(), 
                (100, 140)
            )
            self.jump_sprites.append(sprite)
        
        self.image = self.jump_sprites[0]
        self.character_x_position= 100
        self.character_y_position= 500
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))
        self.y_velocity = 20
        self.y_gravity = 0.5
        self.jump_height = 20
        self.jumping = False

    
    def animate(self):
        if self.jumping:    
            self.current_sprite += 0.5
            if self.current_sprite >= len(self.jump_sprites):
                self.current_sprite = 0 
            self.image = self.jump_sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

        else:
            self.current_sprite += 1
            if self.current_sprite >= len(self.sprites):
                self.current_sprite = 0
            self.image = self.sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

    def player_input(self):
        keys = pygame.key.get_pressed()

        if keys[pygame.K_SPACE] and not self.jumping:
            self.jumping = True
        
        if self.jumping:
            self.character_y_position -= self.y_velocity
            self.y_velocity -= self.y_gravity
            if self.y_velocity < -self.jump_height:
                self.y_velocity = self.jump_height
                self.jumping = False
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))     

    def update(self):
        self.player_input()
        self.animate()
 

class Item(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.character_x_position = x
        self.character_y_position = y

    def update(self):
        self.rect.x -= 7


class Obstacle(Item):
    def __init__(self, item_type):
        super().__init__(x = 1400, y = 525)

        self.item_type = item_type

        if self.item_type == "obstacle 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 01.png').convert_alpha(), (200, 200))
        
        elif self.item_type == "obstacle 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 02.png').convert_alpha(), (200, 200))

        elif self.item_type == "obstacle 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 03.png').convert_alpha(), (200, 200))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


class Reward(Item):
    def __init__(self, item_type):
        super().__init__(x = 1500, y = 500)

        self.item_type = item_type
        
        if self.item_type == "point 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 01.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 02.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 03.png').convert_alpha(), (120, 120))

        elif self.item_type == "point 4":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 05.png').convert_alpha(), (120, 120))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


SCREEN_WIDTH= 1280
SCREEN_HEIGHT = 720

pygame.init() 

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("day 7")

player_object = Player()

player = pygame.sprite.GroupSingle()
player.add(player_object)


obstacle_1= Obstacle(item_type='obstacle 1')
obstacle_2 = Obstacle(item_type='obstacle 2')
obstacle_3 = Obstacle(item_type='obstacle 3')

obstacle_group = pygame.sprite.GroupSingle()
obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3,  obstacle_3)))


reward_1 = Reward(item_type='point 1')
reward_2 = Reward(item_type='point 2')
reward_3 = Reward(item_type='point 3')
reward_4 = Reward(item_type='point 4')

reward_group = pygame.sprite.GroupSingle()
reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))

font = pygame.font.Font(size= 32) #create a font

green = (0, 255, 0)
blue = (0, 0, 128)

running = True

background_img =  pygame.transform.scale(pygame.image.load("./Assets/BG/bg 01.png") , (SCREEN_WIDTH, SCREEN_HEIGHT)) 

clock = pygame.time.Clock()

jumping = False

y_velocity = 20
y_gravity = 1
jump_height = 20

#adding the score and the health variable
SCORE = 0
HEALTH = 100

def collisions():

    global SCORE
    global HEALTH
 
    if pygame.sprite.groupcollide(player, obstacle_group, False, True):
        #When the player collides with the collision then the health goes down
        HEALTH = HEALTH - 10
        obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3,  obstacle_2))
        obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
        obstacle_group.add(obstacle)
        return True 
    
    if obstacle_group.__len__() > 0: 
        if obstacle_group.sprite.rect.x < -100:
            obstacle_group.sprites().pop(0).kill()
            obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3,  obstacle_2))
            obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
            obstacle_group.add(obstacle)
            return True
        
    if reward_group.__len__() > 0: 
        if reward_group.sprite.rect.x < -100:
            reward_group.sprites().pop(0).kill()
            reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
            reward.rect.x = reward.character_x_position + random.randint(0, 260)
            reward_group.add(reward)
            return True
        
        
    if pygame.sprite.groupcollide(reward_group, obstacle_group, False, False):
        obstacle_group.sprite.rect.x += random.randint(100, 500)

    if pygame.sprite.groupcollide(player, reward_group, False, True):
        #When the player collides with the reward then the score goes up
        SCORE = SCORE + 10
        reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
        reward.rect.x = reward.character_x_position + random.randint(0, 260)
        reward_group.add(reward)
    
    return True

def display_score():
    global SCORE
    text_render = font.render(f"Score: {SCORE}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (200, 100)
    screen.blit(text_render, text_rect)

def display_health():
    global HEALTH
    text_render = font.render(f"Health : {HEALTH}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (1100, 100)
    screen.blit(text_render, text_rect)


while running == True:
    
    dt = clock.tick(60) 

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

    keys_pressed = pygame.key.get_pressed()

    if keys_pressed[pygame.K_SPACE]:
        jumping = True

    screen.blit(background_img, (0, 0))  

    player.draw(screen)
    player.update()

    obstacle_group.draw(screen)
    obstacle_group.update()

    reward_group.draw(screen)
    reward_group.update()

    display_score()
    display_health()
    running = collisions()
    

    pygame.display.flip()

MOVE THE BACKGROUND 

In [ ]:

import pygame
import random

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

        self.sprites = []
        self.current_sprite = 0
        for i in range(1, 20):
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Character/Frame {i}.png').convert_alpha(), 
                (100, 150)
            )
            self.sprites.append(sprite)

        self.jump_sprites = []
        for i in range(1, 13):  
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Jump/Jump_frame {i}.png').convert_alpha(), 
                (100, 140)
            )
            self.jump_sprites.append(sprite)
        
        self.image = self.jump_sprites[0]
        self.character_x_position= 100
        self.character_y_position= 500
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))
        self.y_velocity = 20
        self.y_gravity = 0.5
        self.jump_height = 20
        self.jumping = False

    
    def animate(self):
        if self.jumping:    
            self.current_sprite += 0.5
            if self.current_sprite >= len(self.jump_sprites):
                self.current_sprite = 0 
            self.image = self.jump_sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

        else:
            self.current_sprite += 1
            if self.current_sprite >= len(self.sprites):
                self.current_sprite = 0
            self.image = self.sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

    def player_input(self):
        keys = pygame.key.get_pressed()

        if keys[pygame.K_SPACE] and not self.jumping:
            self.jumping = True
        
        if self.jumping:
            self.character_y_position -= self.y_velocity
            self.y_velocity -= self.y_gravity
            if self.y_velocity < -self.jump_height:
                self.y_velocity = self.jump_height
                self.jumping = False
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))     

    def update(self):
        self.player_input()
        self.animate()
 

class Item(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.character_x_position = x
        self.character_y_position = y

    def update(self):
        self.rect.x -= 7


class Obstacle(Item):
    def __init__(self, item_type):
        super().__init__(x = 1400, y = 525)

        self.item_type = item_type

        if self.item_type == "obstacle 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 01.png').convert_alpha(), (200, 200))
        
        elif self.item_type == "obstacle 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 02.png').convert_alpha(), (200, 200))

        elif self.item_type == "obstacle 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 03.png').convert_alpha(), (200, 200))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


class Reward(Item):
    def __init__(self, item_type):
        super().__init__(x = 1500, y = 500)

        self.item_type = item_type

        
        if self.item_type == "point 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 01.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 02.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 03.png').convert_alpha(), (120, 120))

        
        elif self.item_type == "point 4":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 05.png').convert_alpha(), (120, 120))


        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


SCREEN_WIDTH= 1280
SCREEN_HEIGHT = 720

pygame.init() 

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("day 7")


player_object = Player()

player = pygame.sprite.GroupSingle()
player.add(player_object)



obstacle_1= Obstacle(item_type='obstacle 1')
obstacle_2 = Obstacle(item_type='obstacle 2')
obstacle_3 = Obstacle(item_type='obstacle 3')

obstacle_group = pygame.sprite.GroupSingle()
obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_3)))


reward_1 = Reward(item_type='point 1')
reward_2 = Reward(item_type='point 2')
reward_3 = Reward(item_type='point 3')
reward_4 = Reward(item_type='point 4')

reward_group = pygame.sprite.GroupSingle()
reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))

font = pygame.font.Font(size= 32)

green = (0, 255, 0)
blue = (0, 0, 128)

running = True

background_img =  pygame.transform.scale(pygame.image.load("./Assets/BG/bg 01.png") , (SCREEN_WIDTH, SCREEN_HEIGHT)) 

clock = pygame.time.Clock()

jumping = False

y_velocity = 20
y_gravity = 1
jump_height = 20

SCORE = 0

HEALTH = 100

SCROLL = 0 

def collisions():
    global SCORE
    global HEALTH
 
    if pygame.sprite.groupcollide(player, obstacle_group, False, True):
        HEALTH = HEALTH - 10
        obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3,  obstacle_2))
        obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
        obstacle_group.add(obstacle)
        return True
    
    if obstacle_group.__len__() > 0: 
        if obstacle_group.sprite.rect.x < -100:
            obstacle_group.sprites().pop(0).kill()
            obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
            obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
            obstacle_group.add(obstacle)
            return True
        
    if reward_group.__len__() > 0: 
        if reward_group.sprite.rect.x < -100:
            reward_group.sprites().pop(0).kill()
            reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
            reward.rect.x = reward.character_x_position + random.randint(0, 260)
            reward_group.add(reward)
            return True
        
        
    if pygame.sprite.groupcollide(reward_group, obstacle_group, False, False):
        obstacle_group.sprite.rect.x += random.randint(100, 500)

    if pygame.sprite.groupcollide(player, reward_group, False, True):
        SCORE = SCORE + 10
        reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
        reward.rect.x = reward.character_x_position + random.randint(0, 260)
        reward_group.add(reward)
    
    return True

def display_score():
    global SCORE
    text_render = font.render(f"Score: {SCORE}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (200, 100)
    screen.blit(text_render, text_rect)

def display_health():
    global HEALTH
    text_render = font.render(f"Health : {HEALTH}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (1100, 100)
    screen.blit(text_render, text_rect)


while running == True:
    
    dt = clock.tick(60) 

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

    keys_pressed = pygame.key.get_pressed()

    if keys_pressed[pygame.K_SPACE]:
        jumping = True


    #we move the 3 backgrounds
    for i in range(0, 3):
        screen.blit(background_img, (i * SCREEN_WIDTH + SCROLL, 0))
    SCROLL = SCROLL - 5
    
    player.draw(screen)
    player.update()

    obstacle_group.draw(screen)
    obstacle_group.update()

    reward_group.draw(screen)
    reward_group.update()

    display_score()
    display_health()
    running = collisions()
    

    pygame.display.flip()


fixing the screen glitch

In [ ]:
#fixing the screen glitch

import pygame
import random

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

        self.sprites = []
        self.current_sprite = 0
        for i in range(1, 20):
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Character/Frame {i}.png').convert_alpha(), 
                (100, 150)
            )
            self.sprites.append(sprite)

        self.jump_sprites = []
        for i in range(1, 13):  
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Jump/Jump_frame {i}.png').convert_alpha(), 
                (100, 140)
            )
            self.jump_sprites.append(sprite)
        
        self.image = self.jump_sprites[0]
        self.character_x_position= 100
        self.character_y_position= 500
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))
        self.y_velocity = 20
        self.y_gravity = 0.5
        self.jump_height = 20
        self.jumping = False

    
    def animate(self):
        if self.jumping:    
            self.current_sprite += 0.5
            if self.current_sprite >= len(self.jump_sprites):
                self.current_sprite = 0 
            self.image = self.jump_sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

        else:
            self.current_sprite += 1
            if self.current_sprite >= len(self.sprites):
                self.current_sprite = 0
            self.image = self.sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

    def player_input(self):
        keys = pygame.key.get_pressed()

        if keys[pygame.K_SPACE] and not self.jumping:
            self.jumping = True
        
        if self.jumping:
            self.character_y_position -= self.y_velocity
            self.y_velocity -= self.y_gravity
            if self.y_velocity < -self.jump_height:
                self.y_velocity = self.jump_height
                self.jumping = False
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))     

    def update(self):
        self.player_input()
        self.animate()
 


class Item(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.character_x_position = x
        self.character_y_position = y

    def update(self):
        self.rect.x -= 7


class Obstacle(Item):
    def __init__(self, item_type):
        super().__init__(x = 1400, y = 525)

        self.item_type = item_type

        if self.item_type == "obstacle 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 01.png').convert_alpha(), (200, 200))
        
        elif self.item_type == "obstacle 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 02.png').convert_alpha(), (200, 200))

        elif self.item_type == "obstacle 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 03.png').convert_alpha(), (200, 200))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


class Reward(Item):
    def __init__(self, item_type):
        super().__init__(x = 1500, y = 500)

        self.item_type = item_type

        
        if self.item_type == "point 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 01.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 02.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 03.png').convert_alpha(), (120, 120))

        elif self.item_type == "point 4":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 05.png').convert_alpha(), (120, 120))

        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


SCREEN_WIDTH= 1280
SCREEN_HEIGHT = 720

pygame.init() 

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("day 7")

player_object = Player()

player = pygame.sprite.GroupSingle()
player.add(player_object)


obstacle_1= Obstacle(item_type='obstacle 1')
obstacle_2 = Obstacle(item_type='obstacle 2')
obstacle_3 = Obstacle(item_type='obstacle 3')

obstacle_group = pygame.sprite.GroupSingle()
obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3,  obstacle_3)))


reward_1 = Reward(item_type='point 1')
reward_2 = Reward(item_type='point 2')
reward_3 = Reward(item_type='point 3')
reward_4 = Reward(item_type='point 4')

reward_group = pygame.sprite.GroupSingle()
reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))
font = pygame.font.Font(size= 32)

green = (0, 255, 0)
blue = (0, 0, 128)

running = True

background_img =  pygame.transform.scale(pygame.image.load("./Assets/BG/bg 01.png") , (SCREEN_WIDTH, SCREEN_HEIGHT)) 

clock = pygame.time.Clock()

jumping = False

y_velocity = 20
y_gravity = 1
jump_height = 20


SCORE = 0

HEALTH = 100

SCROLL = 0 

def collisions():

    global SCORE
    global HEALTH
 
    if pygame.sprite.groupcollide(player, obstacle_group, False, True):
        HEALTH = HEALTH - 10
        obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3,  obstacle_2))
        obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
        obstacle_group.add(obstacle)
        return True
    
    if obstacle_group.__len__() > 0: 
        if obstacle_group.sprite.rect.x < -100:
            obstacle_group.sprites().pop(0).kill()
            obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
            obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
            obstacle_group.add(obstacle)
            return True
        
    if reward_group.__len__() > 0: 
        if reward_group.sprite.rect.x < -100:
            reward_group.sprites().pop(0).kill()
            reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
            reward.rect.x = reward.character_x_position + random.randint(0, 260)
            reward_group.add(reward)
            return True
        

    if pygame.sprite.groupcollide(reward_group, obstacle_group, False, False):
        obstacle_group.sprite.rect.x += random.randint(100, 500)

    if pygame.sprite.groupcollide(player, reward_group, False, True):
        SCORE = SCORE + 10
        reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
        reward.rect.x = reward.character_x_position + random.randint(0, 260)
        reward_group.add(reward)
    
    return True

def display_score():
   
    global SCORE
    text_render = font.render(f"Score: {SCORE}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (200, 100)
    screen.blit(text_render, text_rect)

def display_health():
    global HEALTH
    text_render = font.render(f"Health : {HEALTH}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (1100, 100)
    screen.blit(text_render, text_rect)

while running == True:
    
    dt = clock.tick(60) 

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

    keys_pressed = pygame.key.get_pressed()

    if keys_pressed[pygame.K_SPACE]:
        jumping = True

    # screen.blit(background_img, (0, 0))  

    #we move the 3 backgrounds
    for i in range(0, 3):
        screen.blit(background_img, (i * SCREEN_WIDTH + SCROLL, 0))
    SCROLL -=5

    #adding this so that when background image gets out of frame we reset it
    if abs(SCROLL)>SCREEN_WIDTH:
        SCROLL = 0
    
    player.draw(screen)
    player.update()

    obstacle_group.draw(screen)
    obstacle_group.update()

    reward_group.draw(screen)
    reward_group.update()

    display_score()
    display_health()
    running = collisions()
    

    pygame.display.flip()


ADDING GAME OVER SCREEN

In [ ]:
import pygame
import random

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

        self.sprites = []
        self.current_sprite = 0
        for i in range(1, 20):
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Character/Frame {i}.png').convert_alpha(), 
                (100, 150)
            )
            self.sprites.append(sprite)

        self.jump_sprites = []
        for i in range(1, 13):  
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Jump/Jump_frame {i}.png').convert_alpha(), 
                (100, 140)
            )
            self.jump_sprites.append(sprite)
        
        self.image = self.jump_sprites[0]
        self.character_x_position= 100
        self.character_y_position= 500
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))
        self.y_velocity = 20
        self.y_gravity = 0.5
        self.jump_height = 20
        self.jumping = False

    
    def animate(self):
        if self.jumping:    
            self.current_sprite += 0.5
            if self.current_sprite >= len(self.jump_sprites):
                self.current_sprite = 0 
            self.image = self.jump_sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

        else:
            self.current_sprite += 1
            if self.current_sprite >= len(self.sprites):
                self.current_sprite = 0
            self.image = self.sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

    def player_input(self):
        keys = pygame.key.get_pressed()

        if keys[pygame.K_SPACE] and not self.jumping:
            self.jumping = True
        
        if self.jumping:
            self.character_y_position -= self.y_velocity
            self.y_velocity -= self.y_gravity
            if self.y_velocity < -self.jump_height:
                self.y_velocity = self.jump_height
                self.jumping = False
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))     

    def update(self):
        self.player_input()
        self.animate()
 


class Item(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.character_x_position = x
        self.character_y_position = y

    def update(self):
        self.rect.x -= 7

class Obstacle(Item):
    def __init__(self, item_type):
        super().__init__(x = 1400, y = 525)

        self.item_type = item_type

        if self.item_type == "obstacle 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 01.png').convert_alpha(), (200, 200))
        
        elif self.item_type == "obstacle 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 02.png').convert_alpha(), (200, 200))

        elif self.item_type == "obstacle 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 03.png').convert_alpha(), (200, 200))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


class Reward(Item):
    def __init__(self, item_type):
        super().__init__(x = 1500, y = 500)

        self.item_type = item_type

        if self.item_type == "point 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 01.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 02.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 03.png').convert_alpha(), (120, 120))

        elif self.item_type == "point 4":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 05.png').convert_alpha(), (120, 120))

        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


SCREEN_WIDTH= 1280
SCREEN_HEIGHT = 720

pygame.init() 

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("day 7")

player_object = Player()

player = pygame.sprite.GroupSingle()
player.add(player_object)



obstacle_1= Obstacle(item_type='obstacle 1')
obstacle_2 = Obstacle(item_type='obstacle 2')
obstacle_3 = Obstacle(item_type='obstacle 3')

obstacle_group = pygame.sprite.GroupSingle()
obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_3)))


reward_1 = Reward(item_type='point 1')
reward_2 = Reward(item_type='point 2')
reward_3 = Reward(item_type='point 3')
reward_4 = Reward(item_type='point 4')

reward_group = pygame.sprite.GroupSingle()
reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))


font = pygame.font.Font(size= 32)
green = (0, 255, 0)
blue = (0, 0, 128)

running = True
jumping = False

game_over = False #this is by default going to be false

background_img =  pygame.transform.scale(pygame.image.load("./Assets/BG/bg 01.png") , (SCREEN_WIDTH, SCREEN_HEIGHT)) 

game_over_background_img =  pygame.transform.scale(pygame.image.load('./Assets/Gameover/Gameover.png').convert(), (SCREEN_WIDTH, SCREEN_HEIGHT))

clock = pygame.time.Clock()

y_velocity = 20
y_gravity = 1
jump_height = 20


SCORE = 0

HEALTH = 100

SCROLL = 0 


def collisions():

    global SCORE
    global HEALTH
 
    if pygame.sprite.groupcollide(player, obstacle_group, False, True):
        # When the player is colliding with the object we check if the health is below zero in which case we will return False
        # this will return false when the health is below 0 and hence we are going to have the game over be true and running be false
        HEALTH = HEALTH - 10
        obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
        obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
        obstacle_group.add(obstacle)
        if HEALTH <= 0:
            return False
        return True 
    
    if obstacle_group.__len__() > 0: 
        if obstacle_group.sprite.rect.x < -100:
            obstacle_group.sprites().pop(0).kill()
            obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
            obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
            obstacle_group.add(obstacle)
            return True
        
    if reward_group.__len__() > 0: 
        if reward_group.sprite.rect.x < -100:
            reward_group.sprites().pop(0).kill()
            reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
            reward.rect.x = reward.character_x_position + random.randint(0, 260)
            reward_group.add(reward)
            return True
        
        
    if pygame.sprite.groupcollide(reward_group, obstacle_group, False, False):
        obstacle_group.sprite.rect.x += random.randint(100, 500)

    if pygame.sprite.groupcollide(player, reward_group, False, True):
        SCORE = SCORE + 10
        reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
        reward.rect.x = reward.character_x_position + random.randint(0, 260)
        reward_group.add(reward)
    
    return True

def display_score():
    global SCORE
    text_render = font.render(f"Score: {SCORE}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (200, 100)
    screen.blit(text_render, text_rect)

def display_health():
    global HEALTH
    text_render = font.render(f"Health : {HEALTH}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (1100, 100)
    screen.blit(text_render, text_rect)


while True:
    
    if game_over == True:
        screen.blit(game_over_background_img, (0, 0))
        text = font.render('Space for restart', True, green, blue)
        textRect = text.get_rect()
        x_position = SCREEN_WIDTH//2
        y_position = SCREEN_HEIGHT//2
        if x_position == SCREEN_WIDTH//2:
            x_position = SCREEN_WIDTH//2 + 0.5 * random.randint(1, 2)
            y_position = SCREEN_HEIGHT//2 + 0.5 * random.randint(1, 2)

        textRect.center = (x_position, y_position)
        screen.blit(text, textRect)


    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

    keys_pressed = pygame.key.get_pressed()

    if keys_pressed[pygame.K_SPACE]:
        jumping = True
        if not running and jumping:
            running = True
            game_over = not(running)
    

    if running == True:
        dt = clock.tick(60) 

        for i in range(0, 3):
            screen.blit(background_img, (i * SCREEN_WIDTH + SCROLL, 0))
        SCROLL -=5

        if abs(SCROLL)>SCREEN_WIDTH:
            SCROLL = 0
        
        player.draw(screen)
        player.update()

        obstacle_group.draw(screen)
        obstacle_group.update()

        reward_group.draw(screen)
        reward_group.update()

        display_score()
        display_health()

        #when the running is False the game over is true
        running = collisions()
        game_over = not(running)
        
    pygame.display.flip()

Fix the health and the score status

In [ ]:
# Ensure health never goes below 0. 
# If health reaches 0 -> trigger Game Over screen.
# If score reaches 100 -> trigger Win screen.

# On restart:
# - Reset health back to full (0 bug fix).
# - Allow restarting the game with SPACE from both 
#   Game Over and Win screens.


import pygame
import random

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

        self.sprites = []
        self.current_sprite = 0
        for i in range(1, 20):
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Character/Frame {i}.png').convert_alpha(), 
                (100, 150)
            )
            self.sprites.append(sprite)

        self.jump_sprites = []
        for i in range(1, 13):  
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Jump/Jump_frame {i}.png').convert_alpha(), 
                (100, 140)
            )
            self.jump_sprites.append(sprite)
        
        self.image = self.jump_sprites[0]
        self.character_x_position= 100
        self.character_y_position= 500
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))
        self.y_velocity = 20
        self.y_gravity = 0.5
        self.jump_height = 20
        self.jumping = False

    
    def animate(self):
        if self.jumping:    
            self.current_sprite += 0.5
            if self.current_sprite >= len(self.jump_sprites):
                self.current_sprite = 0 
            self.image = self.jump_sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

        else:
            self.current_sprite += 1
            if self.current_sprite >= len(self.sprites):
                self.current_sprite = 0
            self.image = self.sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

    def player_input(self):
        keys = pygame.key.get_pressed()

        if keys[pygame.K_SPACE] and not self.jumping:
            self.jumping = True
        
        if self.jumping:
            self.character_y_position -= self.y_velocity
            self.y_velocity -= self.y_gravity
            if self.y_velocity < -self.jump_height:
                self.y_velocity = self.jump_height
                self.jumping = False
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))     

    def update(self):
        self.player_input()
        self.animate()
 


class Item(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.character_x_position = x
        self.character_y_position = y

    def update(self):
        self.rect.x -= 7


class Obstacle(Item):
    def __init__(self, item_type):
        super().__init__(x = 1400, y = 525)

        self.item_type = item_type

        if self.item_type == "obstacle 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 01.png').convert_alpha(), (200, 200))
        
        elif self.item_type == "obstacle 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 02.png').convert_alpha(), (200, 200))

        elif self.item_type == "obstacle 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 03.png').convert_alpha(), (200, 200))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


class Reward(Item):
    def __init__(self, item_type):
        super().__init__(x = 1500, y = 500)

        self.item_type = item_type

        if self.item_type == "point 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 01.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 02.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 03.png').convert_alpha(), (120, 120))

        elif self.item_type == "point 4":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 05.png').convert_alpha(), (120, 120))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


SCREEN_WIDTH= 1280
SCREEN_HEIGHT = 720

pygame.init()

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("day 7")


player_object = Player()

player = pygame.sprite.GroupSingle()
player.add(player_object)



obstacle_1= Obstacle(item_type='obstacle 1')
obstacle_2 = Obstacle(item_type='obstacle 2')
obstacle_3 = Obstacle(item_type='obstacle 3')
obstacle_group = pygame.sprite.GroupSingle()
obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_3)))

reward_1 = Reward(item_type='point 1')
reward_2 = Reward(item_type='point 2')
reward_3 = Reward(item_type='point 3')
reward_4 = Reward(item_type='point 4')

reward_group = pygame.sprite.GroupSingle()
reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))


font = pygame.font.Font(size= 32)
green = (0, 255, 0)
blue = (0, 0, 128)

running = True

jumping = False

game_won = False

game_over = not(running)
background_img =  pygame.transform.scale(pygame.image.load("./Assets/BG/bg 01.png") , (SCREEN_WIDTH, SCREEN_HEIGHT)) 

#add the game over background
game_over_background_img =  pygame.transform.scale(pygame.image.load('./Assets/Gameover/Gameover.png').convert(), (SCREEN_WIDTH, SCREEN_HEIGHT))
game_won_background_img = pygame.transform.scale(pygame.image.load('./Assets/You win/Win (2).png').convert(), (SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()


y_velocity = 20
y_gravity = 1
jump_height = 20

SCORE = 0

HEALTH = 100

SCROLL = 0 

def collisions():

    global SCORE
    global HEALTH
    global game_won
 
    if pygame.sprite.groupcollide(player, obstacle_group, False, True):
        HEALTH = HEALTH - 10
        obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
        obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
        obstacle_group.add(obstacle)
        if HEALTH <= 0:
            return False
        return True 
    if obstacle_group.__len__() > 0: 
        if obstacle_group.sprite.rect.x < -100:
            obstacle_group.sprites().pop(0).kill()
            obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
            obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
            obstacle_group.add(obstacle)
            return True
        
    if reward_group.__len__() > 0: 
        if reward_group.sprite.rect.x < -100:
            reward_group.sprites().pop(0).kill()
            reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
            reward.rect.x = reward.character_x_position + random.randint(0, 260)
            reward_group.add(reward)
            return True
        

    if pygame.sprite.groupcollide(reward_group, obstacle_group, False, False):
        obstacle_group.sprite.rect.x += random.randint(100, 500)

    if pygame.sprite.groupcollide(player, reward_group, False, True):
        SCORE = SCORE + 10
        if SCORE > 100:
            game_won = True
        reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
        reward.rect.x = reward.character_x_position + random.randint(0, 260)
        reward_group.add(reward)
    
    return True

def display_score():
    global SCORE
    text_render = font.render(f"Score: {SCORE}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (200, 100)
    screen.blit(text_render, text_rect)

def display_health():
    global HEALTH
    text_render = font.render(f"Health : {HEALTH}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (1100, 100)
    screen.blit(text_render, text_rect)


while True:
    
    if game_over == True:
        screen.blit(game_over_background_img, (0, 0))
        text = font.render('Space for restart', True, green, blue)
        textRect = text.get_rect()
        x_position = SCREEN_WIDTH//2
        y_position = SCREEN_HEIGHT//2
        if x_position == SCREEN_WIDTH//2:
            x_position = SCREEN_WIDTH//2 + 0.5 * random.randint(1, 2)
            y_position = SCREEN_HEIGHT//2 + 0.5 * random.randint(1, 2)

        textRect.center = (x_position, y_position)
        screen.blit(text, textRect)


    if game_won == True:
        running = False
        screen.blit(game_won_background_img, (0, 0))
        text = font.render('Space for Restart', True, green, blue)
        textRect = text.get_rect()
        x_position = SCREEN_WIDTH//2
        y_position = SCREEN_HEIGHT//2
        textRect.center = (x_position, y_position)
        screen.blit(text, textRect)

    #moved the code for the movement outside of the running block
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

    keys_pressed = pygame.key.get_pressed()

    if keys_pressed[pygame.K_SPACE]:
        jumping = True
        if not running and jumping:
            #this indicates that the game was not in running state and is coming into the running state either from game_over or game_won
            running = True
            game_over = not(running)
            game_won = False
            SCORE = 0
            HEALTH = 100
            player.sprite.rect.y = 550
            
            obstacle_group.sprites().pop(0).kill()
            reward_group.sprites().pop(0).kill()
            reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))
            obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_3)))

    
    if running == True:
        dt = clock.tick(60) 

        for i in range(0, 3):
            screen.blit(background_img, (i * SCREEN_WIDTH + SCROLL, 0))
        SCROLL -=5

        if abs(SCROLL)>SCREEN_WIDTH:
            SCROLL = 0
        
        player.draw(screen)
        player.update()

        obstacle_group.draw(screen)
        obstacle_group.update()

        reward_group.draw(screen)
        reward_group.update()

        display_score()
        display_health()

        running = collisions()
        game_over = not(running)
        
    pygame.display.flip()


Add a start-screen also make the game more difficult my reducing the jump height

In [ ]:
#also make the game more difficult my reducing the jump height

import pygame
import random

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

        self.sprites = []
        self.current_sprite = 0
        for i in range(1, 20):
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Character/Frame {i}.png').convert_alpha(), 
                (100, 150)
            )
            self.sprites.append(sprite)

        self.jump_sprites = []
        for i in range(1, 13):  
            sprite = pygame.transform.scale(
                pygame.image.load(f'./Assets/Jump/Jump_frame {i}.png').convert_alpha(), 
                (100, 140)
            )
            self.jump_sprites.append(sprite)
        
        self.image = self.jump_sprites[0]
        self.character_x_position= 100
        self.character_y_position= 500
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))
        self.y_velocity = 15
        self.y_gravity = 0.5
        self.jump_height = 15
        self.jumping = False

    
    def animate(self):
        if self.jumping:    
            self.current_sprite += 0.5
            if self.current_sprite >= len(self.jump_sprites):
                self.current_sprite = 0 
            self.image = self.jump_sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

        else:
            self.current_sprite += 1
            if self.current_sprite >= len(self.sprites):
                self.current_sprite = 0
            self.image = self.sprites[int(self.current_sprite)]
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))

    def player_input(self):
        keys = pygame.key.get_pressed()

        if keys[pygame.K_SPACE] and not self.jumping:
            self.jumping = True
        
        if self.jumping:
            self.character_y_position -= self.y_velocity
            self.y_velocity -= self.y_gravity
            if self.y_velocity < -self.jump_height:
                self.y_velocity = self.jump_height
                self.jumping = False
            self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))     

    def update(self):
        self.player_input()
        self.animate()
 
class Item(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.character_x_position = x
        self.character_y_position = y

    def update(self):
        self.rect.x -= 10


class Obstacle(Item):
    def __init__(self, item_type):
        super().__init__(x = 1400, y = 525)

        self.item_type = item_type

        if self.item_type == "obstacle 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 01.png').convert_alpha(), (200, 200))
        
        elif self.item_type == "obstacle 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 02.png').convert_alpha(), (200, 200))

        elif self.item_type == "obstacle 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Obstacles/Obstacle 03.png').convert_alpha(), (200, 200))

        
        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


class Reward(Item):
    def __init__(self, item_type):
        super().__init__(x = 1500, y = 500)

        self.item_type = item_type

        if self.item_type == "point 1":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 01.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 2":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 02.png').convert_alpha(), (100, 100))

        elif self.item_type == "point 3":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 03.png').convert_alpha(), (120, 120))

        elif self.item_type == "point 4":
            self.image = pygame.transform.scale(pygame.image.load('./Assets/Points/point 05.png').convert_alpha(), (120, 120))

        self.rect = self.image.get_rect(center = (self.character_x_position, self.character_y_position))


SCREEN_WIDTH= 1280
SCREEN_HEIGHT = 720


pygame.init() 

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("day 7")

player_object = Player()

player = pygame.sprite.GroupSingle()
player.add(player_object)


obstacle_1= Obstacle(item_type='obstacle 1')
obstacle_2 = Obstacle(item_type='obstacle 2')
obstacle_3 = Obstacle(item_type='obstacle 3')

obstacle_group = pygame.sprite.GroupSingle()
obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_3)))

reward_1 = Reward(item_type='point 1')
reward_2 = Reward(item_type='point 2')
reward_3 = Reward(item_type='point 3')
reward_4 = Reward(item_type='point 4')

reward_group = pygame.sprite.GroupSingle()
reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))


font = pygame.font.Font(size= 32)
green = (0, 255, 0)
blue = (0, 0, 128)

running = False #by default this is off now

jumping = False

game_won = False

game_start = True #game start _screen 

game_over = False #this will also be false

background_img =  pygame.transform.scale(pygame.image.load("./Assets/BG/bg 01.png") , (SCREEN_WIDTH, SCREEN_HEIGHT)) 

#add the game over background
game_start_background_img =  pygame.transform.scale(pygame.image.load("./Assets/Starting Screen/Starting screen-2.png").convert(), (SCREEN_WIDTH, SCREEN_HEIGHT))
game_over_background_img = pygame.transform.scale(pygame.image.load('./Assets/Gameover/Gameover.png').convert(), (SCREEN_WIDTH, SCREEN_HEIGHT))
game_won_background_img = pygame.transform.scale(pygame.image.load('./Assets/You win/Win (2).png').convert(), (SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()

y_velocity = 15
y_gravity = 0.5
jump_height = 15

SCORE = 0

HEALTH = 100

SCROLL = 0 #Default value

def collisions():

    global SCORE
    global HEALTH
    global game_won
 
    if pygame.sprite.groupcollide(player, obstacle_group, False, True):
        # When the player is colliding with the object we check if the health is below zero in which case we will return False
        HEALTH = HEALTH - 10
        obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
        obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
        obstacle_group.add(obstacle)
        if HEALTH <= 0:
            return False
        return True 
    if obstacle_group.__len__() > 0: 
        if obstacle_group.sprite.rect.x < -100:
            obstacle_group.sprites().pop(0).kill()
            obstacle = random.choice((obstacle_1, obstacle_2, obstacle_3, obstacle_2))
            obstacle.rect.x = obstacle.character_x_position + random.randint(0, 200)
            obstacle_group.add(obstacle)
            return True
        
    if reward_group.__len__() > 0: 
        if reward_group.sprite.rect.x < -100:
            reward_group.sprites().pop(0).kill()
            reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
            reward.rect.x = reward.character_x_position + random.randint(0, 260)
            reward_group.add(reward)
            return True
        
    
    if pygame.sprite.groupcollide(reward_group, obstacle_group, False, False):
        obstacle_group.sprite.rect.x += random.randint(100, 500)

    if pygame.sprite.groupcollide(player, reward_group, False, True):
        SCORE = SCORE + 10
        if SCORE > 100:
            game_won = True
        reward = random.choice((reward_1, reward_2, reward_3, reward_3, reward_4))
        reward.rect.x = reward.character_x_position + random.randint(0, 260)
        reward_group.add(reward)
    
    return True

def display_score():
    global SCORE
    text_render = font.render(f"Score: {SCORE}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (200, 100)
    screen.blit(text_render, text_rect)

def display_health():
    global HEALTH
    text_render = font.render(f"Health : {HEALTH}", False, blue, green)
    text_rect = text_render.get_rect()
    text_rect.center = (1100, 100)
    screen.blit(text_render, text_rect)


while True:
    #add this code block

    if game_start == True:
        screen.blit(game_start_background_img, (0, 0))
        text = font.render('Space to begin', True, green, blue)
        textRect = text.get_rect()
        x_position = SCREEN_WIDTH//2
        y_position = SCREEN_HEIGHT//2
        if x_position == SCREEN_WIDTH//2:
            x_position = SCREEN_WIDTH//2 + 0.5 * random.randint(1, 2)
            y_position = SCREEN_HEIGHT//2 + 0.5 * random.randint(1, 2)

        textRect.center = (x_position, y_position)
        screen.blit(text, textRect)

    if game_over == True:
        screen.blit(game_over_background_img, (0, 0))
        text = font.render('Space for restart', True, green, blue)
        textRect = text.get_rect()
        x_position = SCREEN_WIDTH//2
        y_position = SCREEN_HEIGHT//2
        if x_position == SCREEN_WIDTH//2:
            x_position = SCREEN_WIDTH//2 + 0.5 * random.randint(1, 2)
            y_position = SCREEN_HEIGHT//2 + 0.5 * random.randint(1, 2)

        textRect.center = (x_position, y_position)
        screen.blit(text, textRect)


    if game_won == True:
        running = False
        screen.blit(game_won_background_img, (0, 0))
        text = font.render('Space for Restart', True, green, blue)
        textRect = text.get_rect()
        x_position = SCREEN_WIDTH//2
        y_position = SCREEN_HEIGHT//2
        textRect.center = (x_position, y_position)
        screen.blit(text, textRect)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

    keys_pressed = pygame.key.get_pressed()

    if keys_pressed[pygame.K_SPACE]:
        jumping = True
        if not running and jumping:
            
            running = True
            game_over = not(running)
            game_won = False
            game_start = False #add this
            SCORE = 0
            HEALTH = 100
            player.sprite.rect.y = 550
            
            obstacle_group.sprites().pop(0).kill()
            reward_group.sprites().pop(0).kill()
            reward_group.add(random.choice((reward_1, reward_2, reward_3, reward_3, reward_4)))
            obstacle_group.add(random.choice((obstacle_1, obstacle_2, obstacle_3,  obstacle_3)))


    if running == True:
        dt = clock.tick(60) 

        #we move the 3 backgrounds
        for i in range(0, 3):
            screen.blit(background_img, (i * SCREEN_WIDTH + SCROLL, 0))
        SCROLL -=5

        if abs(SCROLL)>SCREEN_WIDTH:
            SCROLL = 0
        
        player.draw(screen)
        player.update()

        obstacle_group.draw(screen)
        obstacle_group.update()

        reward_group.draw(screen)
        reward_group.update()

        display_score()
        display_health()

        #when the running is False the game over is true
        running = collisions()
        game_over = not(running)
        
    pygame.display.flip()
